In [1]:
import os
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SQLContext

import math
import itertools
from pyspark.sql.functions import udf
from pyspark.sql.types import *


In [2]:
df = pd.read_csv('C:/Users/arshi/Desktop/ArshiyaUSB/Fall2018/256/Project/CMPE256-Airbnb/Module1/output/rating_prediction_full.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800995 entries, 0 to 800994
Data columns (total 3 columns):
listing_id     800995 non-null int64
reviewer_id    800995 non-null int64
rating         800995 non-null int64
dtypes: int64(3)
memory usage: 18.3 MB


In [4]:
df.nunique()

listing_id      34839
reviewer_id    703051
rating              6
dtype: int64

In [5]:
df.reviewer_id.value_counts()

197711       63
3973614      52
76641978     42
339131       40
2539165      36
101640211    34
39274139     33
563572       33
77978742     32
24667379     32
84558452     30
31402335     30
15355355     30
26152563     28
43382407     28
22585386     28
45139957     28
23545132     27
21726142     26
14438        26
39177007     26
76628146     25
95940868     24
28774703     24
320624       24
5429871      24
109699676    24
86126627     23
77352080     23
17387960     22
             ..
138197010     1
55263049      1
86734363      1
20978485      1
62925619      1
31466290      1
25180977      1
16790320      1
24167214      1
17100542      1
18064194      1
50631481      1
87983072      1
8647496       1
140727132     1
1272667       1
115565402     1
55329984      1
31683416      1
59318762      1
18620222      1
69407572      1
27468626      1
39865320      1
287600        1
1301325       1
82039628      1
73655114      1
66321225      1
41258800      1
Name: reviewer_id, Lengt

In [6]:
df.listing_id.value_counts()

903972      489
891117      474
903947      462
21218       394
26785       383
546383      359
31994       352
58059       351
5178        350
8168619     342
834190      339
467634      336
476983      332
10101135    328
9145202     326
27024       322
1093570     322
557018      320
63360       318
21220       311
847690      308
67208       306
151199      304
999248      298
7675781     298
4261693     297
394474      296
2284027     295
106363      295
1120700     295
           ... 
11548744      1
9282691       1
16072935      1
19589738      1
9959880       1
6233819       1
4838260       1
20811403      1
13410478      1
16381984      1
8742625       1
2744182       1
17517651      1
9944013       1
20933335      1
6688119       1
7827616       1
2365645       1
8014087       1
19751573      1
20662567      1
16441446      1
7922002       1
19163061      1
19117903      1
20571653      1
8814070       1
3323247       1
7365429       1
10163240      1
Name: listing_id, Length

In [7]:
df.listing_id.nunique()

34839

In [8]:
df.rating.value_counts()

5    324276
4    212149
3    119025
0     81108
2     50666
1     13771
Name: rating, dtype: int64

In [9]:
df.shape

(800995, 3)

In [10]:
counts = df['listing_id'].value_counts()
print(counts)
df= df[df['listing_id'].isin(counts[counts >= 10].index)]

903972      489
891117      474
903947      462
21218       394
26785       383
546383      359
31994       352
58059       351
5178        350
8168619     342
834190      339
467634      336
476983      332
10101135    328
9145202     326
27024       322
1093570     322
557018      320
63360       318
21220       311
847690      308
67208       306
151199      304
999248      298
7675781     298
4261693     297
394474      296
2284027     295
106363      295
1120700     295
           ... 
11548744      1
9282691       1
16072935      1
19589738      1
9959880       1
6233819       1
4838260       1
20811403      1
13410478      1
16381984      1
8742625       1
2744182       1
17517651      1
9944013       1
20933335      1
6688119       1
7827616       1
2365645       1
8014087       1
19751573      1
20662567      1
16441446      1
7922002       1
19163061      1
19117903      1
20571653      1
8814070       1
3323247       1
7365429       1
10163240      1
Name: listing_id, Length

In [11]:
df.shape

(735588, 3)

In [12]:
df.describe()

,listing_id,reviewer_id,rating
count,7.355880e+05,7.355880e+05,735588.000000
mean,6.711766e+06,4.153548e+07,3.678351
std,5.706286e+06,3.844911e+07,1.561335
min,2.515000e+03,1.000000e+00,0.000000
25%,1.347847e+06,9.770348e+06,3.000000
50%,5.211902e+06,2.905638e+07,4.000000
75%,1.132511e+07,6.340887e+07,5.000000
max,2.081912e+07,1.526344e+08,5.000000


In [13]:
# Calling spark session to register application
spark = SparkSession \
    .builder \
    .appName("Recom") \
    .config("spark.recom.demo", "1") \
    .getOrCreate()

In [14]:
"""
Loading and Parsing Dataset

""" 
ratings = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("C:/Users/arshi/Desktop/ArshiyaUSB/Fall2018/256/Project/CMPE256-Airbnb/Module1/output/rating_prediction_full.csv")

In [15]:
ratings


DataFrame[listing_id: int, reviewer_id: int, rating: int]

In [16]:
data = ratings.select('reviewer_id','listing_id','Rating')
data.show(5)

+-----------+----------+------+
|reviewer_id|listing_id|Rating|
+-----------+----------+------+
|    6279455|   9452127|     5|
|   23770684|   9452127|     2|
|   29070567|   9452127|     5|
|   58381183|   9452127|     3|
|   29014080|   9452127|     5|
+-----------+----------+------+
only showing top 5 rows



In [17]:
data.printSchema()

root
 |-- reviewer_id: integer (nullable = true)
 |-- listing_id: integer (nullable = true)
 |-- Rating: integer (nullable = true)



In [18]:
data.describe().show()

+-------+-------------------+-----------------+------------------+
|summary|        reviewer_id|       listing_id|            Rating|
+-------+-------------------+-----------------+------------------+
|  count|             800995|           800995|            800995|
|   mean|4.230238320549816E7|7180466.618531951|3.6731240519603743|
| stddev|3.903449681193699E7|5966198.629587632| 1.573049482796602|
|    min|                  1|             2515|                 0|
|    max|          152696826|         21154538|                 5|
+-------+-------------------+-----------------+------------------+



In [19]:
data.groupBy("listing_id").count().sort("count",ascending=False).show()


+----------+-----+
|listing_id|count|
+----------+-----+
|    903972|  489|
|    891117|  474|
|    903947|  462|
|     21218|  394|
|     26785|  383|
|    546383|  359|
|     31994|  352|
|     58059|  351|
|      5178|  350|
|   8168619|  342|
|    834190|  339|
|    467634|  336|
|    476983|  332|
|  10101135|  328|
|   9145202|  326|
|     27024|  322|
|   1093570|  322|
|    557018|  320|
|     63360|  318|
|     21220|  311|
+----------+-----+
only showing top 20 rows



Use the SparkML ALS algorithm to train a model to provide recommendations. The mandatory parameters to the ALS algorithm are the columns that identify the users, the items, and the ratings. 
Run the fit() method to train the model

In [20]:
als =  ALS(maxIter=25,rank=12,regParam=0.2,nonnegative=True, userCol="reviewer_id", itemCol="listing_id",ratingCol="Rating",coldStartStrategy="drop")
model = als.fit(data)


Run the transform() method to score the model and output a DataFrame with an additional prediction column that shows the predicted rating

In [21]:
predictions = model.transform(data)

predictions.show()

+-----------+----------+------+----------+
|reviewer_id|listing_id|Rating|prediction|
+-----------+----------+------+----------+
|    1642299|      5803|     3| 2.8548274|
|    4139979|      5803|     5| 4.7580457|
|    1306432|      5803|     5| 4.7580457|
|    8341565|      5803|     4| 3.8064368|
|   93182128|      5803|     5| 4.7140603|
|    2131239|      5803|     5| 4.7580457|
|   21282389|      5803|     5| 4.7580457|
|    4213410|      5803|     5| 4.7580457|
|     374347|      5803|     5| 4.7580457|
|    2293395|      5803|     4| 3.8064368|
|     898248|      5803|     5| 4.7580457|
|    2191623|      5803|     5| 4.7580457|
|      19091|      5803|     5| 4.7580457|
|     254645|      5803|     5|  4.731411|
|    9834544|      5803|     3| 2.8548274|
|    7220467|      5803|     0|       0.0|
|      39478|      5803|     5| 4.7580457|
|      39478|      5803|     5| 4.7580457|
|      42873|      5803|     5| 4.7580457|
|     281328|      5803|     5|   4.41099|
+----------

Using the RegressionEvaluator method to compare continuous values with the root mean squared calculation.

In [22]:
# Evaluate the model by computing RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

print ('RMSE is %s' % rmse)

RMSE is 0.2531079401293451


To accurately evaluate the model,we need to split the data set between a training data set,validation and test set.

I used cross validator from spark ML, but as it takes forever to tune parameters, I am directly using the best parameters obtained from tuning.
Using parameters obtained from parameter tuning, train model and predict ratings on validation data

In [ ]:
'''
(trainingData, validationData, testData) = data.randomSplit([0.8,0.1,0.1])

evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating", predictionCol="prediction")

als =  ALS(userCol="reviewer_id", itemCol="listing_id",ratingCol="Rating")

paramGrid = ParamGridBuilder().addGrid(als.rank, [5,12,16]).addGrid(als.maxIter, [20,25]).addGrid(als.regParam, [0.1, 0.2]).addGrid(als.nonnegative, [True,False]).build()

crossval = CrossValidator(estimator=als, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

cvModel = crossval.fit(trainingData)
predictions = cvModel.transform(validationData)

print "The root mean squared error for our model is: " + str(evaluator.evaluate(predictions.na.drop()))
'''

In [25]:
(trainingData, validationData, testData) = data.randomSplit([0.8,0.1,0.1])

als = ALS(maxIter=25,rank=12,regParam=0.2,nonnegative=True, userCol="reviewer_id", itemCol="listing_id",ratingCol="Rating",coldStartStrategy="drop")
cvModel = als.fit(trainingData)
pred = cvModel.transform(validationData)
    
# Evaluate the model by computing RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating",predictionCol="prediction")
rmse = evaluator.evaluate(pred)

print ('RMSE is %s' % rmse)

RMSE is 2.006486542689759


using k fold cross validation

In [48]:
from pyspark.sql import SQLContext
sc = spark.sparkContext
def kfoldALS(data, k=3, userCol="reviewer_id", itemCol="listing_id", ratingCol="Rating", metricName="rmse"):
    evaluations = []
    weights = [1.0] * k
    splits = data.randomSplit(weights)
    for i in range(0, k):  
        testingSet = splits[i]
        trainingSet = spark.createDataFrame(sc.emptyRDD(), data.schema)
        for j in range(0, k):
            if i == j:
                continue
            else:
                trainingSet = trainingSet.union(splits[j])
        als = ALS(maxIter=25,rank=12,regParam=0.2,nonnegative=True, userCol=userCol, itemCol=itemCol, ratingCol=ratingCol,coldStartStrategy="drop")
        model = als.fit(trainingSet)
        predictions = model.transform(testingSet)
        evaluator = RegressionEvaluator(metricName=metricName, labelCol="Rating", predictionCol="prediction")
        evaluation = evaluator.evaluate(predictions.na.drop())
        print ("Loop " + str(i+1) + ": " + metricName + " = " + str(evaluation))
        evaluations.append(evaluation)
    return sum(evaluations)/float(len(evaluations))

In [49]:
print ("RMSE = " + str(kfoldALS(data, k=4)))

Loop 1: rmse = 2.0297479458878156
Loop 2: rmse = 2.0229908198227515
Loop 3: rmse = 2.013526223477445
Loop 4: rmse = 2.024762763678099
RMSE = 2.022756938216528


In [ ]:
userRecs = model.recommendForAllUsers(3)

In [55]:
userRecs.where(userRecs.reviewer_id == 53).select("recommendations.listing_id", "recommendations.Rating").collect()

[Row(listing_id=[16693621, 12432739, 20900984], Rating=[4.76357889175415, 4.466307640075684, 4.2514967918396])]

In [58]:
type(userRecs)

pyspark.sql.dataframe.DataFrame

In [33]:
from pyspark.sql.functions import lit

def recommendListings(model, user, n):
    # Create a Spark DataFrame with the specified user and all the listings listed in the data DataFrame
    dataSet = data.select("listing_id").distinct().withColumn("reviewer_id", lit(user))

    # Create a Spark DataFrame with the listings that have already been rated by this user
    AlreadyRated = data.filter(data.reviewer_id == user).select("listing_id", "reviewer_id")

    # Apply the recommender system to the data set without the already rated listings to predict ratings
    predictions = model.transform(dataSet.subtract(AlreadyRated)).dropna().orderBy("prediction", ascending=False).limit(n).select("listing_id", "prediction")

    predictions.show(truncate=False)

In [56]:
user = 1
rec = recommendListings(cvModel,user,3)

print ("Top 3 listings recommended for:%d" % user)


+----------+----------+
|listing_id|prediction|
+----------+----------+
|17986966  |4.990666  |
|16693621  |4.9841533 |
|16425205  |4.8267455 |
+----------+----------+

Top 3 listings recommended for:1


In [ ]:
temp_path = "C:/Users/arshi/Desktop/ArshiyaUSB/Fall2018/256/Project/CMPE256-Airbnb/Module2"
als_path = temp_path + "/als"
als.save(als_path)
als2 = ALS.load(als_path)


In [ ]:
model_path = temp_path + "/als_model"
model.save(model_path)
model2 = ALSModel.load(model_path)
